In [1]:
%display latex
%runfile character.py

# Exemples de calculs

### Cas du diagramme 013

In [2]:
v = vandermonde(Diagram([(0,0),(1,0),(3,0)]))
v.factor()

(x01 - x02) * (-x00 + x01) * (x00 - x02) * (x00 + x01 + x02)

On crée le sous-espace engendré par $\mathcal{V}_\gamma$ et ses dérivées partielles. On peut ajouter les opérateurs de Steenrod sans changement du résultat.

In [3]:
generator = {v.multidegree() : [v]}
list_op = merge(merge(partial_derivatives(v.parent()), steenrod_operators(1, 1)), steenrod_operators(1, 2))

In [4]:
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1.basis()

{(0,): (1,),
 (1,): (x00, x01, x02),
 (2,): (x00^2 - x02^2, x01^2 - x02^2, -x00*x01 + x00*x02, -x00*x01 + x01*x02),
 (3,): (1/3*x00^3 - 1/3*x01^3 - x00*x02^2 + x01*x02^2,
  -1/3*x00^3 + x00*x01^2 - x01^2*x02 + 1/3*x02^3,
  -x00^2*x01 + 1/3*x01^3 + x00^2*x02 - 1/3*x02^3),
 (4,): (-x00^3*x01 + x00*x01^3 + x00^3*x02 - x01^3*x02 - x00*x02^3 + x01*x02^3,)}

In [5]:
character(IsotypicComponent(W1, 3))

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + 2*s[2] # s[2, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Appliquons $\sum_i \partial_{x_i}$ et $\sum_i x_i \partial_{x_i}^2$ à $\mathcal{V}_{\gamma}$. 

In [6]:
v.symmetric_derivative((1,))

-3*x00^2*x01 + 3*x00*x01^2 + 3*x00^2*x02 - 3*x01^2*x02 - 3*x00*x02^2 + 3*x01*x02^2

In [7]:
v.steenrod_op(0, 2)

-6*x00^2*x01 + 6*x00*x01^2 + 6*x00^2*x02 - 6*x01^2*x02 - 6*x00*x02^2 + 6*x01*x02^2

Polarisation

In [8]:
op_pol = polarization_operators(2)
W2 = PolarizedSpace(IsotypicComponent(W1, 3), op_pol)
character(W2)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + 2*s[2] # s[2, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

In [9]:
r=2
list_degrees = [tuple(k1 if j==i1 else 0 for j in range(r)) 
                for k1 in range(1, 3+1) for i1 in range(0, r)]
#list_degrees += [(1,1),(1,2),(1,3),(2,1),(2,2),(2,3),(3,1),(3,2),(3,3)]
sym_diff = symmetric_derivatives(r, list_degrees)
#sym_diff = merge(merge(symmetric_derivatives(r, list_degrees), 
#                       steenrod_operators(r, 1)), steenrod_operators(r, 2))
character(Range(W2, sym_diff))

s[] # s[3] + s[1] # s[2, 1] + s[2] # s[2, 1] + s[3] # s[1, 1, 1]

In [10]:
charac = character(W2) - character(Range(W2, sym_diff))
charac

s[1] # s[3] + s[2] # s[2, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Ou dans la base des monomiales

In [11]:
character(W2, left_basis=m) - character(Range(W2, sym_diff), left_basis=m)

m[1] # s[3] + m[1, 1] # s[2, 1] + m[2] # s[2, 1] + m[2, 1] # s[2, 1] + m[2, 2] # s[1, 1, 1] + m[3] # s[2, 1] + m[3, 1] # s[1, 1, 1] + m[4] # s[1, 1, 1]

On factorise le caractère par représentation de $S_3$.

In [12]:
factorise(charac, 3)

[1, 1, 1]


s[4]

[3]


s[1]

[2, 1]


s[2] + s[3]

### Autres cas

Pas de quotient dans ce cas.

In [13]:
compute_character(Partition([2]))

s[] # s[2] + s[1] # s[1, 1]

In [14]:
compute_character(Partition([3,1]))

s[] # s[3, 1] + s[1] # s[2, 1, 1] + s[1] # s[2, 2] - s[1, 1] # s[2, 1, 1] + s[2] # s[2, 1, 1] - s[2, 1] # s[1, 1, 1, 1] + s[3] # s[1, 1, 1, 1]

In [15]:
compute_character(Partition([1,1,1]))

s[] # s[1, 1, 1]

In [16]:
compute_character(Partition([2,1,1]))

s[] # s[2, 1, 1] + s[1] # s[1, 1, 1, 1]